# Recommender systems

One of the most common uses of big data is to predict and suggest what users may want. 
This allows Google to show you relevant ads or to suggest news in Google Now; Amazon to recommend relevant products; Netflix to recommend movies that you might like; or most recently, the famous **Weekly Dicovery** of Spotify.

All these products are based on systems of recommendation: a information retrieval method to provide users with relevant, yet novel and diverse, information. 

In this class we will use a pretty famous dataset based on movies ratings, 'MovieLens', to learn the basics of recommender systems. 

Table of Contents (times are approximated)

1. [Getting and analysing some data (1.5-2 h, typically until break)](#data)
2. [Most popular movies (0.5-1 min)](#popular)
3. [Metrics for recommender systems (2-2.5h)](#metrics)

#  Movix.ai

- Usa primero popularidad.
- Después, tu escoges una película y ya te va clusterianzado (le gusta el amor). 
- Después también se puede basar en contenido. 

#  Sesgo de los sistemas de recomendación (Filter Bubble)

Por ejemplo, si ves algo de teoría de la conspiración, te salen sólo vídeos de eso.

En el fonde es porque sólo estamos tomando la medida de relevancia. Pero debeŕiamos también en cuenta el novelty (métricas que midan la novedad, para que el pequeño cantante salga adelante) o la diversidad (que no sean sólo notificas de El Pais o sólo de LA Razón). 

A mí se me ocurre clusterizar vídeos. A veces a lo mejor estoy en modo facha hoy pero otro día esoy en modo música antigua.

# Algunas ideas clave (JL)

- Recomendación de TFM: crear un enviroment de Python, será más fácil para compartir, llevar a producción, etc.
- Los sistemas de recomendación son muy subjetivos.
- Para un sistema de recomendación necesitamos mucho dato. El primer ejemplo es de pega
- En sistemas de recomendación no hace falta hacer un gran algoritmo, aunque los hay cojocomplicados
- La distribución en sistemas de recomendación suele ser la long tail (hay unos pocos productos muy conocidos y muchos poco conocidos:

![a ver](https://ai2-s2-public.s3.amazonaws.com/figures/2017-08-08/7d20fe87fb0e483a633bdc22ff9f91a8dea4ed8b/2-Figure1-1.png)

La gracia de estas distribuciones es que cogiendo sólo unos cuantos items (los famosos, de la izquierda, llamados soft tail) te dan el 50% del negocio. Sell more of less. Amazon, Youtube, Spotify es esto.

Para explotar esto con un sistema de recomendaciónlo que tengo que conseguir es ir mostrando cosas del Long Tail parecidos a cosas que ya triunfan y están en el Soft Tail. 

- En estas clases vamos a usar diccionarios y matrices. 

# Feedback explícito

* Rating que da una persona a algo (peli, etc.), opiniopnes, encuestas, likes. 
* Tiene un sesgo
* Menos volumen. 
* Lo bueno de este es que tiene positivo y negativo. El implícito no. 

Históricamente se ha trabajdo con esto, porque nació antes de la era del Big Data y porque además se confiaba en que era importante saber el like / dislike. 

Netflix Prize: dieron 1M al que mejore su algoritmo un 10%. Pero ellos tenían ratings explícito, así que por eso la gente investigó eso. También Netflix usasba el RMSE planteándolo como un problema de regresión. Ahora ya se ha descartado totalmente el RMSE
    
# Feedback implícito:

* Conteo de gente que ha dado el rating, tiemp que esas personas lo han estado viendo, número de compras, clicks...)
* Tenemos muuuuucho dato, sin que la gente lo dé. 
* Es objetivo, pero interpretable (que yo compre una peli no quiere decir que me haya gustado)        
* En estos casos hablamos de confianza (si la he visto varias veces) y preferencia (binaria, 1 o 0, si he ido tengo un 1)

# Sistemas de recomendación: ¿son supervisados o no?

Netflix intentó precedir el rating que un usuario daría. Eso es una regresión (supervisado). Pero se demostró que lo imporante no era dar una valoración, sino dar una lista de sugerencias al usuario. Ya está demostrado que esto no se puede hacer así con RMSE y regresión.

Luego se pasó al mundo de la clasificación. Usando preferencia y confianza. Por eso Netflix pasó a Like y Dislike. Preferencia es binaria => problema de clasificación . Pero ahí tendremo problemas: el 1 es fácil de sacar. El 0 cómo lo sacas? Seríe todo el catálogo no visto?. 

Se está tendiendo ahora a in supervisado ni no supervisado, se tienen a RL (Reinformecent -Learning). Resueve el problema de explore (mostrar cosas desconocidas)/exploit (enseñar cosas que sabemos que le va a gustar casi seguro). 


In [167]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import io
import os

<a id='data'></a>
## 1.1 Load data

We will use MovieLens dataset, which is one of the most common datasets used when implementing and testing recommender engines. This data set consists of:
* 100,000 ratings (1-5) from 943 users on 1682 movies. 
* Each user has rated at least 20 movies. 
* Simple demographic info for the users (age, gender, occupation, zip)

The data was collected through the MovieLens [website](https://movielens.org) during the seven-month period from September 19th, 
1997 through April 22nd, 1998. This data has been cleaned up - users
who had less than 20 ratings or did not have complete demographic
information were removed from this data set.

You can download the dataset [here](http://files.grouplens.org/datasets/movielens/ml-100k.zip).

Take a look at the readme file!!!

In [168]:
data_root = "data/"

In [169]:
!cat {data_root}README

SUMMARY & USAGE LICENSE

MovieLens data sets were collected by the GroupLens Research Project
at the University of Minnesota.
 
This data set consists of:
	* 100,000 ratings (1-5) from 943 users on 1682 movies. 
	* Each user has rated at least 20 movies. 
        * Simple demographic info for the users (age, gender, occupation, zip)

The data was collected through the MovieLens web site
(movielens.umn.edu) during the seven-month period from September 19th, 
1997 through April 22nd, 1998. This data has been cleaned up - users
who had less than 20 ratings or did not have complete demographic
information were removed from this data set. Detailed descriptions of
the data file can be found at the end of this file.

Neither the University of Minnesota nor any of the researchers
involved can guarantee the correctness of the data, its suitability
for any particular purpose, or the validity of results based on the
use of the data set.  The data set may be used for any research
purposes under th

**Esto es un dataset de pega** 

Para un sistema de recomendación necesitamos mucho dato. 

Además, no vamos a usar los géneros de las películas. 

Hay dos **familias** de sistemas de reocmendación:

* Basados en contenido. Se usan cuando no tienes histórico del usuario. Lo que haces es un perfil y buscar perfiles similares. Esto ocurre mucho, así que también es importante
* Basados en histórico del usuario. En el que más se ha invertido en algoritmia. La clase la vamos a centrar en estos. Dentro de estos hablaremos de un subtipo: **filtrado colaborativo** (el filtrado depende de histórico de muchos usuarios). 

In [170]:
! head data/u.data

196	242	3	881250949
186	302	3	891717742
22	377	1	878887116
244	51	2	880606923
166	346	1	886397596
298	474	4	884182806
115	265	2	881171488
253	465	5	891628467
305	451	3	886324817
6	86	3	883603013


Está separado por tabuladores

In [171]:
columns = ['user_id', 'item_id', 'rating','timestamp' ]
datafile = os.path.join(data_root, "u.data")
data = pd.read_csv(datafile, sep='\t', names=columns)
data.head()

,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


### A remainder of the numpy library

* **Pandas** library is nothing alse than **numpy** under the hood (numpy with steroids, if you like). You can access the data (in matrix from) with he "values" attribute, e.g. data.values*

In [172]:
data.values

array([[      196,       242,         3, 881250949],
       [      186,       302,         3, 891717742],
       [       22,       377,         1, 878887116],
       ...,
       [      276,      1090,         1, 874795795],
       [       13,       225,         2, 882399156],
       [       12,       203,         3, 879959583]])

In [173]:
# The attribute shape provides the shape of the matrix


In [174]:
data.values.shape

(100000, 4)

In [175]:
# Note that if we return the first column, we get a vector (of 100000 components)


In [176]:
data.values[:,0]

array([196, 186,  22, ..., 276,  13,  12])

In [177]:
# same with the first row (this time, we get a vector of 4 components)


In [178]:
data.values[0,:]

array([      196,       242,         3, 881250949])

In [179]:
# access all rows, and first 3 columns 


In [180]:
# access all collumns, and first 3 rows 


In [181]:
# access first 10 rows

# This is equivalent to data.values[:10]

In [182]:
# access first column


In [183]:
# Number of users and items
n_users = data.user_id.unique().size

In [184]:
# Number of items
# Con numpy , por complicarnos un poco
n_items = np.unique(data.values[:,1]).shape[0]

In [185]:
print("There are %s users and %s items" %(n_users, n_items))

There are 943 users and 1682 items


## 1.2 A dictionary for movies and a search tool

In order to analyze the predicted recommendations, let's create a python dictonary that will allow us to translate any item id to the corresponding movie title. Also, let's write a small function that returns the ids of the movies containing some text.

Cuando imprimamos las recomendaciones nos saldrá: para este usuarios recomendamos la 12, la 34 y la 35. Para tener la correspondencia con la película en sí usaremos diccionarios. 

The correspondance between titles and ids is stored in the u.item file

Leermos la cabecera de el fichero u.item donde guardamos la corredpondencia entre títulos e ids. 

Además tiene esos 1 y 0s que son géneros de cada película. Esto no lo vamos a usar. Se usaría para sistema de recpomendación basado en contenido

In [186]:
items_id_file = os.path.join(data_root, "u.item")
!head $items_id_file

1|Toy Story (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Toy%20Story%20(1995)|0|0|0|1|1|1|0|0|0|0|0|0|0|0|0|0|0|0|0
2|GoldenEye (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?GoldenEye%20(1995)|0|1|1|0|0|0|0|0|0|0|0|0|0|0|0|0|1|0|0
3|Four Rooms (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Four%20Rooms%20(1995)|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|1|0|0
4|Get Shorty (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Get%20Shorty%20(1995)|0|1|0|0|0|1|0|0|1|0|0|0|0|0|0|0|0|0|0
5|Copycat (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Copycat%20(1995)|0|0|0|0|0|0|1|0|1|0|0|0|0|0|0|0|1|0|0
6|Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)|01-Jan-1995||http://us.imdb.com/Title?Yao+a+yao+yao+dao+waipo+qiao+(1995)|0|0|0|0|0|0|0|0|1|0|0|0|0|0|0|0|0|0|0
7|Twelve Monkeys (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Twelve%20Monkeys%20(1995)|0|0|0|0|0|0|0|0|1|0|0|0|0|0|0|1|0|0|0
8|Babe (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Babe%20(1995)|0|0|0|0|1

### *Simple reminder of dictionaries*

In [187]:
aux = {'hola': 'que haces?', 1: '237'}

Vemos que permite que la clave sea cada una de un tipo

In [188]:
# Access value of key='hola'
aux['hola']

'que haces?'

In [189]:
# create new key
aux['adios'] = 'ciao'
aux

{1: '237', 'adios': 'ciao', 'hola': 'que haces?'}

In [190]:
# Update value of existing key
aux['hola'] = 'holarrrr'
aux

{1: '237', 'adios': 'ciao', 'hola': 'holarrrr'}

Además, los diccionarios vienen **indexados**

In [191]:
# get all keys 
list(aux.keys())

['hola', 1, 'adios']

Son pesados en Python 3 y en lugar de devolverte una lista normal te devuelve un objeto. Ponienod list antes ya lo ves como lista. 

In [192]:
# get all values
aux.values()

dict_values(['holarrrr', '237', 'ciao'])

In [193]:
# get all keys and values at the same time
aux.items()

dict_items([('hola', 'holarrrr'), (1, '237'), ('adios', 'ciao')])

Create a dictionary for movie titles and ids

In [194]:
item_dict = {}
with io.open(items_id_file, 'rb') as f:
    for line in f.readlines():
        record = line.split(b'|')
        item_dict[int(record[0])] = str(record[1])
    
item_dict
  

{1: "b'Toy Story (1995)'",
 2: "b'GoldenEye (1995)'",
 3: "b'Four Rooms (1995)'",
 4: "b'Get Shorty (1995)'",
 5: "b'Copycat (1995)'",
 6: "b'Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)'",
 7: "b'Twelve Monkeys (1995)'",
 8: "b'Babe (1995)'",
 9: "b'Dead Man Walking (1995)'",
 10: "b'Richard III (1995)'",
 11: "b'Seven (Se7en) (1995)'",
 12: "b'Usual Suspects, The (1995)'",
 13: "b'Mighty Aphrodite (1995)'",
 14: "b'Postino, Il (1994)'",
 15: 'b"Mr. Holland\'s Opus (1995)"',
 16: "b'French Twist (Gazon maudit) (1995)'",
 17: "b'From Dusk Till Dawn (1996)'",
 18: "b'White Balloon, The (1995)'",
 19: 'b"Antonia\'s Line (1995)"',
 20: "b'Angels and Insects (1995)'",
 21: "b'Muppet Treasure Island (1996)'",
 22: "b'Braveheart (1995)'",
 23: "b'Taxi Driver (1976)'",
 24: "b'Rumble in the Bronx (1995)'",
 25: "b'Birdcage, The (1996)'",
 26: "b'Brothers McMullen, The (1995)'",
 27: "b'Bad Boys (1995)'",
 28: "b'Apollo 13 (1995)'",
 29: "b'Batman Forever (1995)'",
 30: "b'Belle de jou

We can use this dict to see the films a user has seen, for instance. 

In [195]:
for record in data.values[:20]:
    print("User {u} viewed '{m}' and gave a {r} rating".format(
        u=record[0], m=item_dict[record[1]], r=record[2]))  

User 196 viewed 'b'Kolya (1996)'' and gave a 3 rating
User 186 viewed 'b'L.A. Confidential (1997)'' and gave a 3 rating
User 22 viewed 'b'Heavyweights (1994)'' and gave a 1 rating
User 244 viewed 'b'Legends of the Fall (1994)'' and gave a 2 rating
User 166 viewed 'b'Jackie Brown (1997)'' and gave a 1 rating
User 298 viewed 'b'Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1963)'' and gave a 4 rating
User 115 viewed 'b'Hunt for Red October, The (1990)'' and gave a 2 rating
User 253 viewed 'b'Jungle Book, The (1994)'' and gave a 5 rating
User 305 viewed 'b'Grease (1978)'' and gave a 3 rating
User 6 viewed 'b'Remains of the Day, The (1993)'' and gave a 3 rating
User 62 viewed 'b'Men in Black (1997)'' and gave a 2 rating
User 286 viewed 'b"Romy and Michele's High School Reunion (1997)"' and gave a 5 rating
User 200 viewed 'b'Star Trek: First Contact (1996)'' and gave a 5 rating
User 210 viewed 'b'To Wong Foo, Thanks for Everything! Julie Newmar (1995)'' and gave a 3 

In [196]:
# Define a function that retrieves all the ids and titles for movies containing 'text' in its title
def returnItemId(text, ids):
    """
    :param text: string to be looked for in movies titles
    :param ids: dicttionary of {id:title}
    
    :return: a list of (id,title) if text found in titles, and an empty list otherwise.
    """
    # convert input text to lowercase
    text_ = text.lower()
    # find occurances
    search = []
    for id_,title in ids.items():
        if text in title.lower():
            search.append( (id_, title))
    
    return search

## IMPORTANTE

Poner documentación en las funciones, porque con esto siguiente podremos verla después

In [197]:
returnItemId??

In [198]:
returnItemId('but', item_dict)

[(240, "b'Beavis and Butt-head Do America (1996)'"),
 (435, "b'Butch Cassidy and the Sundance Kid (1969)'"),
 (580,
  "b'Englishman Who Went Up a Hill, But Came Down a Mountain, The (1995)'"),
 (1401, "b'M. Butterfly (1993)'"),
 (1459, "b'Madame Butterfly (1995)'"),
 (1614, "b'Reluctant Debutante, The (1958)'"),
 (1621, "b'Butterfly Kiss (1995)'"),
 (1645, "b'Butcher Boy, The (1998)'"),
 (1650, "b'Butcher Boy, The (1998)'")]

**Problema** : hay duplicados

## 1.3 Data consistency (always double check everything!!!)

In [199]:
# check whether titles are unique or not. They are not!!!
len(item_dict.keys())

1682

Aquí vemos el número de películas que teenemos en el array, por id. 

In [200]:
len(item_dict.values())

1682

OJO, aquí este len, no sé cómo, python ya te quita los duplicados. Vamos a usar como lo hace él:

In [201]:
len(set(item_dict.keys()))


1682

In [202]:
len(set(item_dict.values()))

1664

### One work around: create another dict that consolidates ids with the same movie title

Alternatively, you could create a Dataframe from the above dictionary of ids:titles

<pre><code>
dd = pd.DataFrame.from_dict(item_dict, orient='index').reset_index()
dd.columns = ['item_id', 'title']
dd.head()
</code></pre>

find the duplicates (`dd.groupby('title')...`), and create a new id for the items. 

OJO: Además, porque esto lo vamos a mter luego en matrices, necesitamos que los ids estén consecutivos. 

In [203]:
duplicates_item_dict = {}
# Las claves en "duplicates_item_dict" son los nombres de las películas
# Los valores son una lista de los ids (que pueden ser uno solo, o varios)
for id_, name in list(item_dict.items()):
    # clave: name; valor: list(id_)
    ?

# show the duplicated titles
for k,v in list(duplicates_item_dict.items()):
    if len(v)>1:
        print(k,v)

Object `` not found.
Object `` not found.
Object `` not found.
Object `` not found.
Object `` not found.
Object `` not found.
Object `` not found.
Object `` not found.
Object `` not found.
Object `` not found.
Object `` not found.
Object `` not found.
Object `` not found.
Object `` not found.
Object `` not found.
Object `` not found.
Object `` not found.
Object `` not found.
Object `` not found.
Object `` not found.
Object `` not found.
Object `` not found.
Object `` not found.
Object `` not found.
Object `` not found.
Object `` not found.
Object `` not found.
Object `` not found.
Object `` not found.
Object `` not found.
Object `` not found.
Object `` not found.
Object `` not found.
Object `` not found.
Object `` not found.
Object `` not found.
Object `` not found.
Object `` not found.
Object `` not found.
Object `` not found.
Object `` not found.
Object `` not found.
Object `` not found.
Object `` not found.
Object `` not found.
Object `` not found.
Object `` not found.
Object `` not

Object `` not found.
Object `` not found.
Object `` not found.
Object `` not found.
Object `` not found.
Object `` not found.
Object `` not found.
Object `` not found.
Object `` not found.
Object `` not found.
Object `` not found.
Object `` not found.
Object `` not found.
Object `` not found.
Object `` not found.
Object `` not found.
Object `` not found.
Object `` not found.
Object `` not found.
Object `` not found.
Object `` not found.
Object `` not found.
Object `` not found.
Object `` not found.
Object `` not found.
Object `` not found.
Object `` not found.
Object `` not found.
Object `` not found.
Object `` not found.
Object `` not found.
Object `` not found.
Object `` not found.
Object `` not found.
Object `` not found.
Object `` not found.
Object `` not found.
Object `` not found.
Object `` not found.
Object `` not found.
Object `` not found.
Object `` not found.
Object `` not found.
Object `` not found.
Object `` not found.
Object `` not found.
Object `` not found.
Object `` not

Object `` not found.
Object `` not found.
Object `` not found.
Object `` not found.
Object `` not found.
Object `` not found.
Object `` not found.
Object `` not found.
Object `` not found.
Object `` not found.
Object `` not found.
Object `` not found.
Object `` not found.
Object `` not found.
Object `` not found.
Object `` not found.
Object `` not found.
Object `` not found.
Object `` not found.


Create a dict where the key are the original ids, and the values are the unique one. 
We will use this dictionary to remove duplicates in a dataframe.

In [204]:
unique_id_item_dict ={}
for new_id, old_id_list in enumerate(duplicates_item_dict.values()):
    # key: old_id; value: new_id
    ?

unique_id_item_dict = {old_id: new_id for new_id, old_id_list in enumerate(duplicates_item_dict.values()) 
                                      for old_id in old_id_list}

In [205]:
unique_id_item_dict

{}

Create another dict mapping moving titles to this new unique id

In [206]:
unique_item_dict = {unique_id_item_dict[k]:v 
                    for k,v in item_dict.items()}
    
assert(len(set(unique_item_dict.keys())) == 
       len(set(unique_item_dict.values())))

KeyError: 1

Now we can use our `returnItemId()` mehtod safely =)

In [ ]:
returnItemId('but', unique_item_dict)

In [ ]:
returnItemId('but', item_dict)

## Copiamos de notebook de mañana (la solución)

In [ ]:
class MyFirstAwesomeClass():
    def __init__(self):
        pass

In [ ]:
class Rueda():
    def __init__(self,marca,diametro):
        self.marca = marca
        self.diametro = diametro
        self.distancia_recorrida = 0
    def rodar(self,distancia): # no olvidar poner el self. Si no, no aparecerá
           self.distancia_recorrida += distancia

Creamos una instancia

In [ ]:
r1 = Rueda('Michelín', 50)

In [ ]:
r1.marca

In [ ]:
r1.diametro

In [ ]:
r2 = Rueda('Dunlop', 33)
r2.marca

In [ ]:
r1.rodar(10)
r1.rodar(20)

r1.distancia_recorrida


Copiado de la clase de mañana:

In [ ]:
class movielens_100k(object):
    """
    This python class read the movielens-100k dataset and:
        . Preprocess train and test sets
        . Create a python dict that allows us to translate any item id to the corresponding movie title
    """
    
    def __init__(self, data_root, item_id_file='u.item', train_file='u2.base', test_file='u2.test', 
                 columns=['user_id', 'item_id', 'rating', 'timestamp']):
                
        self.data_root = data_root
        self.items_id_file = os.path.join(self.data_root, item_id_file)
        self.train_file = train_file
        self.test_file = test_file
        self.columns = columns
        self.__data = {}  # dictionary to store train and test data
            
    def __getitem__(self, dataset):
        """
        Get train and test data

        :param dataset: dataset name (String)
        :return: pandas dataframe
        """
        return self.__data[dataset]

    def __setitem__(self, key, value):
        """
        Store train and test data in the dictionary self.__data.

        :param key: name of the dataset (String)
        :param value: pandas dataframe
        """
        self.__data[key] = value
        
    def read_data(self, file, verbose=False):
        """
        Read file returning a pandas dataframe
        :param file: file path
        :param verbose: print the first rows of the data
        """
        datafile = os.path.join(self.data_root, file)
        data = pd.read_csv(datafile, sep='\t', names=self.columns)
        if verbose:
            print(data.head())
        return data
            
    def returnItemId(self, text, dict_ids):
        """
        Retrieves all the ids and titles for movies containing 'text' in its title
        :param text: string to be looked for in movies titles
        :param dict_ids: dicttionary of {id:title}

        :return: a list of (id,title) if text found in titles, and an empty list otherwise.
        """
        # convert input text to lowercase
        text_ = text.lower()
        # find occurances
        search = [(k, v.lower().find(text_)) 
                  for k,v in list(dict_ids.items())]

        # Get the IDs corresponding to the given text
        index = [k for k,v in search if v>-1]

        # Return a list with the id and the name
        out = []
        for i in index:
            out.append((i, dict_ids[i]))
        return out
    
    def check_data_consistency(self):
        """
        Check if one film has two or more ids
        """
        self.item_dict = {}
        with io.open(self.items_id_file, 'rb') as f:
            for line in f.readlines():
                record = line.split(b'|')
                self.item_dict[int(record[0])] = str(record[1])
                
        if len(set(self.item_dict.keys())) == len(set(self.item_dict.values())):
            self.unique_item_dict_ = self.item_dict
            return True
        else:
            self.duplicates_item_dict = {}
            for id_,name in list(self.item_dict.items()):
                if name not in self.duplicates_item_dict:
                    self.duplicates_item_dict[name] = [id_]
                else:
                    self.duplicates_item_dict[name] = self.duplicates_item_dict[name]+[id_]

            self.unique_id_item_dict ={}
            for i, lista_id in enumerate(self.duplicates_item_dict.values()) :
                for key in lista_id:
                    self.unique_id_item_dict[key] = i

            self.unique_item_dict_ = {self.unique_id_item_dict[k]:v 
                                for k,v in self.item_dict.items()}
            return False
    
    def correcting_non_unique_movies(self, data_df):
        """
        Preprocess data_df in case the check_data_consistency function fails
        """
        if not self.check_data_consistency():
            data_df['item_id'] = data_df['item_id'].apply(lambda x: self.unique_id_item_dict[x])
            
        return data_df
            
    def train(self, verbose=False):
        """
        Train data
        """
        self['train_set'] = self.read_data(self.train_file, verbose)
        self['train_set'] = self.correcting_non_unique_movies(self['train_set'])
        items = self['train_set'].item_id.unique()
        self.train_id_dict = {j:i for i,j in enumerate(items)}
        self['train_set']['item_id'] = self['train_set']['item_id'].apply(lambda x: self.train_id_dict[x])
        self.unique_item_dict = {self.train_id_dict[v]:self.unique_item_dict_[v] 
                                for v in items}
        self.n_users = self['train_set'].user_id.unique().shape[0]
        self.n_items = self['train_set'].item_id.unique().shape[0]
        self.n_pairs = self['train_set'].shape[0]
        if verbose:
            print('There are %s users, %s items and %s pairs in the test set' \
                  %(self.n_users, self.n_items, self.n_pairs))
    
    def test(self, verbose=False):
        """
        Test data
        """
        self['test_set'] = self.read_data(self.test_file, verbose)
        self['test_set']['item_id'] = self['test_set']['item_id'].apply(lambda x: self.unique_id_item_dict[x])
        self['test_set']['item_id'] = self['test_set']['item_id'].apply(lambda x: self.train_id_dict[x] if x in self.train_id_dict else -1)
        self['test_set'] = self['test_set'][self['test_set']['item_id']!=-1]
        if verbose:
            print('There are %s users, %s items and %s pairs in the test set' \
                  %(self['test_set'].user_id.unique().shape[0], self['test_set'].item_id.unique().shape[0], 
                    self['test_set'].shape[0]))
            
    def get_data(self, verbose=False):
        """
        A kind of main function to return train and test data together
        """
        self.train(verbose)
        self.test(verbose)

Lo instanciamos y probamos a llamar a un método

In [ ]:
movielens = movielens_100k(data_root)

In [ ]:
movielens.check_data_consistency()

Este método es como el princinpal (yo los habría puesto privados a los demás so son auxiliares). Además, podríamos haberlo llamado en el constructor (eso discutible)


In [ ]:
movielens.get_data()

In [ ]:
movielens['test_set'].head()

## 1.4 Train and test sets

GroupLens provides several splits of the dataset, so that we can check the goodness of our algorithms. See the README file for more  details. Here we will use one of such splits.

Please notice that we have to correct for the non-unique movie's id issue!!

In [ ]:
!ls $data_root

In [ ]:
trainfile = os.path.join(data_root, 'ua.base')
!head $trainfile

In [ ]:
columns = ['user_id', 'item_id', 'rating', 'timestamp']
trainfile = os.path.join(data_root, "ua.base")
train = ?
print('There are %s users, %s itmes and %s pairs in the train set' \
      %(train.user_id.unique().shape[0], train.item_id.unique().shape[0], train.item_id.count()))
train.head()


In [ ]:
# same for test
columns = ['user_id', 'item_id', 'rating', 'timestamp']
testfile = os.path.join(data_root, "ua.test")
test = ?
print('There are %s users, %s itmes and %s pairs in the test set' \
      %(test.user_id.unique().shape[0], test.item_id.unique().shape[0], test.item_id.count()))
test.head()


### Correcting for non-unique movies id 

Use Pandas `apply` method. Note it can be applied to all rows in a column, thus returning a Pandas Series; or it can be applied row-wise, he

*Reminder of lambda functions in Python: is a way of calling short functions (1 line of code), without having to define the function in a separted cell.*

In [ ]:
train['item_id'] = ?

In [ ]:
test['item_id'] = ?

<a id='popular'></a>
## 2. Most popular movies

Modelos basados en popularidad. Si le gusta a la gente, gustará. En principio 

http://ir.ii.uam.es/pubs/sigir2018.pdf

Recommending popular items is a simple, yet quite effective baseline for recommendation. Indeed, most RS suffer from a strong *popularity bias*, i.e. they tend to recommend popular items more frequently than they should -just because suggesting what is popular is effective!-. There is a lot of research  devote to understand this behaviour and to develop recipies to avoid it. 

Movies can be ranked according to different popularity metrics:
* Most rated movie (it is assumed that this is the most watched movie)
* Most positively rated movie (rating > 4.0)
* Highest rated movie

## 2.1 Most rated movie

In [ ]:
m1= movielens

In [ ]:
train = m1['train_set']

In [ ]:
test = m1['test_set']

In [ ]:
# group the train dataset by item and count the number of users using Pandas
mostRated = train.groupby('item_id')['user_id'].count()
mostRated.head()

In [ ]:
mostRated.head()

In [ ]:
# sort in descending order
mostRatedSorted = mostRated.sort_values(ascending=False)

In [ ]:
mostRatedSorted.head()

In [ ]:
# add the movie title, using the apply method, returning a Pandas Series in each row
def add_title(row):
    id_ = row[0]
    title_ = m1.unique_item_dict[id_]
    freq_ = row[1]
    new_row = [id_,title_,freq_]
    return pd.Series(new_row, index=['id', 'title', 'freq'])

mostRatedMovies = (mostRatedSorted
                   .reset_index()  #esto sólo par convertilo en data_frame, proque al hacer group_by_ya no es dataset (cosas de pandas)
                   .apply(add_title, axis=1)
                  )

No va por mi máquina virtual

In [ ]:
mostRatedMovies.head()

Aquí sólo hemos puesto las que tienen más popularidad en el sentido de que tienen muchas valoraciones. Pero nos está dando igual unas que otra

## 2.2 Most positively rated movie

Use the query method to filter values in train, `train.query('rating>4')`, or the traditional form `train[train.rating>4]`.

In [ ]:
mostRatedPositively = train.query('rating>=4').groupby('item_id')['user_id'].count()
mostRatedPositively.head()

In [ ]:
# sort in descending order
mostRatedPositivelySorted = mostRatedPositively.sort_values(ascending=False)

In [ ]:
mostRatedPositivelyMovies = (mostRatedPositivelySorted
                   .reset_index()  #esto sólo par convertilo en data_frame, proque al hacer group_by_ya no es dataset (cosas de pandas)
                   .apply(add_title, axis=1)
                  )

mostRatedPositivelyMovies.head()

Más bonito, tipo Linq:

In [ ]:
(train.query('rating >=4')
.groupby('item_id')['user_id']
.count()
.sort_values(ascending=False)
.reset_index()
.apply(add_title,axis= 1)
.head()
)

## 2.3 Highest mean rating movie

In [ ]:
# obtain the highest rated movies, with a min numer of user/ratings
min_ratings =  50

In [ ]:
train = m1['train_set']

In [ ]:
train.groupby('item_id')['rating']

In [ ]:
train.groupby('item_id')['rating'].agg(lambda x : list(x))

In [210]:
(train
.groupby('item_id')['rating']
.agg(lambda x : list(x))
.apply(lambda l: np.mean(l) if len(l) >= min_ratings else 0.0)
.sort_values(ascending=False)
.reset_index()
).head()

,item_id,0
0,93,4.538462
1,252,4.512821
2,816,4.511364
3,136,4.490196
4,410,4.470270


In [207]:
(train
.groupby('item_id')['rating']
.agg(lambda x : list(x))
.apply(lambda l: np.mean(l) if len(l) >= min_ratings else 0.0)
.sort_values(ascending=False)
.reset_index()
.apply(add_title,axis= 1)
.head()
)

KeyError: (4.538461538461538, 'occurred at index 0')

<div class  = "alert alert-info"> 
** QUESTION **: set the value of *min_ratings* to 1, and re-run the cell. What happens now? Change this value
</div>

In [213]:
# Versión de Muiguel, que sí me va
# obtain the highest rated movies, with a minium number of users/ratings.
min_ratings = 200

mostPositiveRated = train.groupby('item_id')['rating'].agg(["count",sum])

mostPositiveRated = mostPositiveRated[mostPositiveRated["count"] >= min_ratings]

mostPositiveRated["freq"] = mostPositiveRated["sum"] / mostPositiveRated["count"] 

aux = (mostPositiveRated["freq"].sort_values(ascending = False)
                   .reset_index()
                   .apply(add_tile, axis=1)
                  )
aux

,id,title,freq
0,252.0,"b""Schindler's List (1993)""",4.512821
1,49.0,"b'Shawshank Redemption, The (1994)'",4.466667
2,7.0,"b'Usual Suspects, The (1995)'",4.395122
3,36.0,b'Star Wars (1977)',4.364425
4,105.0,"b'Godfather, The (1972)'",4.301205
5,283.0,"b""One Flew Over the Cuckoo's Nest (1975)""",4.269231
6,79.0,"b'Silence of the Lambs, The (1991)'",4.264331
7,140.0,b'Raiders of the Lost Ark (1981)',4.261905
8,246.0,b'Titanic (1997)',4.246429
9,139.0,"b'Empire Strikes Back, The (1980)'",4.210356


<div class  = "alert alert-info"> 
** QUESTION **: Which method is better?? How to measure a recommender system? 
</div>

<div class  = "alert alert-info"> 
** IMPORTANT QUESTION **: When might be useful to recommend popular items?
</div>

<a id='metrics'></a>
## 3. Metrics for recommender systems

Los sistemas de recomendacíon realmente te peudes enterar si van bien cuando lo pones en producción (online). Pero aún aśi tenemos métricas para evaluar offline si es apropiado.


As we have seen, even with the simplest solution --aka, recommending popular items-- is difficult to known which technique performs better. For this, there are a number of metrics that allow one to measure the goodness of a recommender system. 

Metrics can be design for measuring the relevance or accuracy of a recommendation, but they can be created for evaluating the novelty of a recommendation, or its diversity. 

For now, we will focus on relevance and accuracy. Several metrics exist:
* Accuracy: rmse, mae.
* Not ranked: Recall@k, Precision@k.
* With rank disccount: map@k, ndcg@k.
* With rank ordering: mean percentile rank.

We will be definiing some of them whitin this class. For the moment, let's talk about precision and recall.

## 3.1 Precision and recall

Lo que recomiendas es el círculo. Lo de la izquierda es lo relevate, lo de la derecha es irrelevante.

El problema en la recomendación es que los true positives los puedes reconocer, ok. Pero los false positives, los he recomendado pero no ha gustado no lo puedo saber (hblando de feedback implícito) 

No podemos medir falsos positivos => No podemos usar Precisión, por tanto tiraremos sólo de recall. 

<img src="https://upload.wikimedia.org/wikipedia/commons/2/26/Precisionrecall.svg" alt="Precision and Recall in IR" style="float: right; width: 300px"/>

The concept of precision and recall comes form the world of information retrieval, have a look at the wikipedia:

https://en.wikipedia.org/wiki/Precision_and_recall

From this entry:

 * "**precision** (also called positive predictive value) is the fraction of retrieved instances that are relevant".
 * "**recall** (also known as sensitivity) is the fraction of relevant instances that are retrieved".

<br />
<div class  = "alert alert-info"> 
** QUESTION **: how do we know if some movie, unknown to the user, is relevant?
</div>

In other words, we cannot measure a false positive --something recommended that was not relevant--. In this regard, only recall-oriented metrics have an actual meaning in RS. Nonetheless, its common practice to define both metrics in RS as follows:
 
### $$\mathrm{recall}@N = \frac{\sum_{k=1}^N rel(k)}{\sum_{i\in \mathcal{I}_u} 1}$$
### $$\mathrm{precision}@N = \frac{\sum_{k=1}^N rel(k)}{N}$$

Here, $\mathcal{I}_u$ is the set of items adopted by user $u$, and $rel(k)$ is the relevance of a recommendation at position k in the list of recommendations. For ratings, the relevance could be defined as those movies rated above a certain threshold, e.g. $r_{ui}>4.0$. 

**Important to note: since precision is pretty much the same as recall in RS, metrcis usch as the *area under the ROC curve* doesn't have any meaning!!**

<div class = "alert alert-success">
As an example, consider a user that watched the following films (este es el test):
<br /><br />
'Designated Mourner, The (1997)'
<br />
'Money Talks (1997)'
<br />
'Madame Butterfly (1995)'
<br />
'Batman Forever (1995)'
<br /><br />
The recommended items were: 
<br /><br />
'Batman (1989)' 
<br />
'Madame Butterfly (1995)'
<br /><br />
**What would be the recall and precision @1? and @2?**
<br />
**What do you think of recommending Batman? Is a bad or a good recommendation?**
</div>

Recal de batman es el número de batman que estuvo en la de test entre el número de películas en test (0 / 4). Ojo, 0 porque está Batman forever. 

r@1 = 0/4

r@2 = 0 / 4v + 1/4

r@3 = (ya no tienen sentido, porque nuestra lista es de 2, pero lo normal es tener una lissta muy grande)

Necestio Recomendación, Test, Número de elementos de los que quiero calcular recall. PEro también necesito saber si la recomendación es buena si NO me ha metido películas ya vistas por el usuario (es decir, las que están ene l conjkunto de training). Por tanto tb necesito el conjunto de training.  

Please notice that there isn't any actual difference between precision and recall in the context of RS: both measure the relevance of the recommendations, and tell nothing about items recommended that haven't been adopted by the user. Thus, it make sense to define a normalized recall as:

### $$\mathrm{recall}@N = \frac{\sum_{i=1}^N rel_i}{\mathrm{min}(N, \sum_{i\in \mathcal{I}_u} 1})$$

This way, results are normalized to 1 always.

<div class="alert alert-success">
**Exercise** Implement the above definition of recall
</div>

**Una recomendacíon es una lista ordenada. Lo vamos a intentar comparar con el conjunto de test. Miraremos si lo que yo he mostrado en la lista se obsevó en el histórico del usuario en test**

Hay otros método más avanzados a ver si los órdenes coinciden. 

Este método me permite o no pasar el conkunto de train. Ese conjunto de train se quitará de la lista de recomendación

In [217]:
def recall_at_n(N, test, recommended, train=None):
    """
    :param N: number of recommendations
    :param test: list of movies seen by user in test
    :param train: list of movies seen by user in train. This has to be removed from the recommended list 
    :param recommended: list of movies recommended
    
    :return the recall
    """
    # para quitarlo, habíamos pensado en usar teoría de conjuntos, pero claro, perdería la ordenación
    if train is not None: # Remove items in train
        rec_true = []
        for r in recommended:
            if r not in train:
                rec_true.append(r)
    else:
        rec_true = recommended    
    intersection = len(set(rec_true[:N]) & set(test)) # intersecciones usar teoría de conjuntos, es decir, usar sets, vamos. 
    return intersection / float(np.minimum(N, len(test))) #es una forma de normallizar

In [218]:
seen = ['Designated Mourner, The (1997)', 'Money Talks (1997)', 'Madame Butterfly (1995)', 'Batman Forever (1995)']
recommended = ['Batman (1989)', 'Madame Butterfly (1995)']

In [219]:
recall_at_n(1, seen, recommended)

0.0

In [221]:
recall_at_n(2, seen, recommended)

0.5

In [222]:
# Check it's well normalized
print(recall_at_n(3, seen, recommended))
print(recall_at_n(10, seen, recommended))
print(recall_at_n(100, seen, recommended))

0.3333333333333333
0.25
0.25


### Now, use this implementation to measure the efficiency of the popularity baselines in the test set. Use the top-5 movies, for instance

In [223]:
mostRatedMovies[:5,1:]

TypeError: unhashable type: 'slice'

In [ ]:
positiveRatedMovies[:5,1:]

In [ ]:
meanRateMovies[:5,1:]

In [ ]:
train.head()

*Since `recall_at_n` takes both train and test list per user, we need to create a dataset with the list of movies seen in train and test*

Thus, get the list of movies per user in train and test, and join the two dataframes. For the join, use the pandas method `merge`.

In [ ]:
# get movies in train per user. For this, group by user and get a list of item ids.
trainUsersGrouped = ?


In [ ]:
# same with test data
testUsersGrouped = ?

In [ ]:
# make the join: use pandas merge method
joined = ?

In [ ]:
joined.head()

In [ ]:
joined.item_id_test.head()

In [ ]:
# How would you access values in test?
?

In [ ]:
# This second method is easier if we want to access several columns at once, and operate over them.
# For instance, if we like to concatenate both train and test list, we will do:
?

In [ ]:
# Use the above method to calculate the recall of the mostRatedMovies recommendation, for each user:
?

*As you can see, some users have a quite large recall (0.5), while for others is small (e.g, 0.14). Let's calculate the mean.*

In [ ]:
topN = 30
# calculate the average recall across all users for mostRatedMovies recommendation
recall_per_user = ?
recall_per_user.mean()

In [ ]:
# calculate the average recall across all users for positiveRatedMovies recommendation
?

In [ ]:
# calculate the average recall across all users for meanRatedMovies recommendation
?

## 3.2 Mean Averaged Precision (MAP) -- Advanced material

Previous metrics did not account for the ranking of the recommendation, i.e. the relative position of a movie within the sorted list of recommendations. **But orders matters!** Metrics like MAP, MRR or NDCG try to tackle down this problem. 

From the blog *http://fastml.com/what-you-wanted-to-know-about-mean-average-precision/*:

> Here’s another way to understand average precision. Wikipedia says AP is used to score document retrieval. You can think of it this way: you type something in Google and it shows you 10 results. It’s probably best if all of them were relevant. If only some are relevant, say five of them, then it’s much better if the relevant ones are shown first. It would be bad if first five were irrelevant and good ones only started from sixth, wouldn’t it? AP score reflects this.

Implementation taken from:

https://github.com/benhamner/Metrics/blob/master/Python/ml_metrics/average_precision.py



## Average Precision 

The Average Precision is definied as:

### $$\mathrm{AP}@N = \frac{\sum_{k=1}^N P(k) \times rel(k)}{\mathrm{min}(N, \sum_{i\in \mathcal{I}_u} 1)}$$

where $P(k)$ is the precision at cut-off in the item list, i.e. the ratio of the number of recommended items adopted, up to the position k, over the number k. Thus:

### $$\mathrm{AP}@N = \frac{\sum_{k=1}^N \left(\sum_{i=1}^k rel(i)\right)/k \times rel(k)}{\mathrm{min}(N, \sum_{i\in \mathcal{I}_u} 1)}$$



<div class = "alert alert-success">
Following the example above, consider a user that watched the following films:
<br /><br />
'Designated Mourner, The (1997)'
<br />
'Money Talks (1997)'
<br />
'Madame Butterfly (1995)'
<br />
'Batman Forever (1995)'
<br /><br />
The recommended items were: 
<br /><br />
'Batman (1989)' 
<br />
'Madame Butterfly (1995)'
<br /><br />

<div class = "alert alert-success">
**Calculate AP@1**
<br /><br />
First, *rel(1)=0*, because Batman was not viewed. Also, *P(1) = 0*. Thus, AP@1=0.
<br />
**Calculate AP@2**
<br /><br />
As before, *rel(1)=0*, so the first term does not contribute. For the second term, *rel(2)=1*, so that *P(2)=0.5*. The numerator is hence:
<br /><br />
$P(1)*rel(1)+P(2)*rel(2)=0*0+0.5*1$
<br /><br />
For the denominator, $N=2$ and $\sum_{i\in \mathcal{I}_u} 1)=4$, thus:
<br /><br />
AP@2 = 0.5/2 = 0.25
</div>

Let's now implement it =)

In [ ]:
def apk(N, test, recommended, train=None):
    """
    Computes the average precision at N given recommendations.
    
    :param N: number of recommendations
    :param test: list of movies seen by user in test
    :param train: list of movies seen by user in train. This has to be removed from the recommended list 
    :param recommended: list of movies recommended
    
    :return The average precision at N over the test set
    """
    if train is not None: 
        rec_true = []
        for r in recommended:
            if r not in train:
                rec_true.append(r)
    else:
        rec_true = recommended    
    predicted = rec_true[:N] # top-k predictions
    
    score = 0.0 # This will store the numerator
    num_hits = 0.0 # This will store the sum of rel(i)

    for i,p in enumerate(predicted):
        if p in test and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits/(i+1.0)

    return score / min(len(test), N)

In [ ]:
seen = ['Designated Mourner, The (1997)', 'Money Talks (1997)', 'Madame Butterfly (1995)', 'Batman Forever (1995)']
recommended = ['Madame Butterfly (1995)', 'Batman (1989)']

In [ ]:
apk(1, seen, recommended)

In [ ]:
apk(2, seen, recommended)

In [ ]:
apk(3, seen, recommended)

## MAP

Mean avergae precision is nothing else than the AP averaged across users ;)

Apply it to popularity baselines

In [ ]:
?

<div class="alert alert-success">
The rest of the class is covered in a different notebook
</div>